# <font color='blue'> MSiA 422 Project 1
## Written by Parth Patel

### IMPORTS

In [23]:
import pandas as pd
import glob
import os

## <font color='orange'> Exercise 1

#### read in data from all states

In [203]:
path = 'namesbystate' # needs to be updated for anyone who uses this code
all_files = glob.glob(os.path.join(path, "*.txt"))

dframes = []
for item in all_files:
    df = pd.read_csv(item,header=None,names=['State','Gender','birth_year','first_name','name_count'])
    dframes.append(df)

combined_df = pd.concat(dframes, axis=0, ignore_index=True)

In [319]:
class BabyNames:
    '''
    FILL OUT DOCSTRING BB
    '''
    #part 2a
    def __init__(self,path):
        all_files = glob.glob(os.path.join(path, "*.txt"))
        dframes = []
        for item in all_files:
            data = pd.read_csv(item,header=None,names=['State','Gender','birth_year','first_name','name_count'])
            dframes.append(data)
        self.df = pd.concat(dframes, axis=0, ignore_index=True)
        
    #part 2b
    def Count(self,state=None,year=None):
        if ((state!=None)&(year!=None)):
            out_df = self.df[(self.df.State==state.upper())&(self.df.birth_year == year)]
        
        elif ((state==None)&(year!=None)):
            out_df = self.df[self.df.birth_year == year]
        
        else:
            out_df = self.df
            
        if out_df.empty:
            print("There were no results for this combination of inputs. There may have been an error with your inputs. Please provide two letter abbrev for states in form of a string and the year as a 4 digit integer. Not providing an arg for either or both is acceptable as well.")
        else:
            print('Total number of births for this combination of inputs: ')
            return len(out_df)
        
    #part 2c:
    def Top10BabyNames(self,state=None,year=None):
        
        self.rank_df = pd.DataFrame()
        self.rank_df['Rank'] = list(range(1,11))
        
        if ((state!=None)&(year!=None)):
            tmp_df = self.df[(self.df.State==state.upper())&(self.df.birth_year == year)]
            self.rank_df['Male'] = tmp_df[tmp_df['Gender']=='M'].sort_values(by='name_count',ascending=False).reset_index()[:10]['first_name']
            self.rank_df['Female'] = tmp_df[tmp_df['Gender']=='F'].sort_values(by='name_count',ascending=False).reset_index()[:10]['first_name']
        
        
        elif ((state==None)&(year!=None)):
            tmp_df = self.df[self.df.birth_year == year]
            self.rank_df['Male'] = tmp_df[tmp_df['Gender']=='M'].sort_values(by='name_count',ascending=False).reset_index()[:10]['first_name']
            self.rank_df['Female'] = tmp_df[tmp_df['Gender']=='F'].sort_values(by='name_count',ascending=False).reset_index()[:10]['first_name']
        else:
            tmp_df = self.df
            tmp_df = tmp_df.groupby(['first_name','Gender']).sum().reset_index()
            self.rank_df['Male'] = tmp_df[tmp_df['Gender']=='M'].sort_values(by='name_count',ascending=False).reset_index()[:10]['first_name']
            self.rank_df['Female'] = tmp_df[tmp_df['Gender']=='F'].sort_values(by='name_count',ascending=False).reset_index()[:10]['first_name']
        
        if self.rank_df.empty:
            print("There were no results for this combination of inputs (state: {}, year: {}). There may have been an error with your inputs. Please provide two letter abbrev for states in form of a string and the year as a 4 digit integer. Not providing an arg for either or both is acceptable as well.".format(state,year))
        else:
            print('Top 10 male & female names for this combination of inputs (state: {}, year: {}):'.format(state,year))
            return self.rank_df
        
    #part 2d:
    def ChangeOfPopularity(self,fromYear, toYear, top):
        fromYr_df = self.df[self.df.birth_year == fromYear].groupby(['first_name']).agg({'name_count':'sum'})
        toYr_df = self.df[self.df.birth_year == toYear].groupby(['first_name']).agg({'name_count':'sum'})
        self.popularity_df = pd.merge(fromYr_df,toYr_df,how='outer',on='first_name').fillna(0)
        self.popularity_df['pop_change'] = self.popularity_df['name_count_x']- self.popularity_df['name_count_y']
        self.popularity_df['pop_change'] = self.popularity_df['pop_change'].astype(int)
        self.popularity_df.sort_values(by='pop_change',ascending=False,inplace=True)
        self.popularity_df.reset_index(inplace=True)
        self.pop_inc = self.popularity_df[:top][['first_name','pop_change']]
        self.pop_dec = self.popularity_df[-top:][['first_name','pop_change']].sort_values(by='pop_change')
        self.pop_same = self.popularity_df[self.popularity_df['pop_change']==0]['first_name']
        
        return self.pop_inc,self.pop_dec,self.pop_same
        
        
        
        
        

In [320]:
baby = BabyNames('namesbystate')

In [321]:
baby.Count(state='IL',year=1993)

Total number of births for this combination of inputs: 


2955

In [322]:
baby.Top10BabyNames('IL',year=2015)

Top 10 male & female names for this combination of inputs (state: IL, year: 2015):


,Rank,Male,Female
0,1,Noah,Olivia
1,2,Liam,Emma
2,3,Alexander,Sophia
3,4,Jacob,Mia
4,5,Michael,Isabella
5,6,Benjamin,Ava
6,7,William,Sofia
7,8,Daniel,Abigail
8,9,Mason,Emily
9,10,James,Amelia


In [323]:
baby.ChangeOfPopularity(2005,2006,20)

(   first_name  pop_change
 0       Emily        2545
 1        Ryan        1547
 2       Tyler        1414
 3    Nicholas        1401
 4     Jessica        1301
 5     Zachary        1278
 6        Emma        1231
 7    Samantha        1176
 8     Michael        1161
 9     Matthew        1143
 10     Andrew        1009
 11      Jacob         998
 12    Madison         961
 13     Ashley         955
 14     Joshua         937
 15      Megan         910
 16      Grace         900
 17      Jenna         852
 18     Sydney         852
 19      Haley         834,       first_name  pop_change
 11258    Addison       -4657
 11257        Ava       -3326
 11256   Isabella       -3039
 11255      Aiden       -2497
 11254     Landon       -2467
 11253       Noah       -2438
 11252    Tristan       -1827
 11251      Jayla       -1650
 11250      Logan       -1625
 11249      Keira       -1567
 11248     Jayden       -1541
 11247      Diego       -1471
 11246      Alexa       -1450
 11245     Au